# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-03-09 21:55:02--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.003s  

2025-03-09 21:55:02 (7.37 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-03-09 21:55:03--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [7]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 3, 6, 21, 55, 5, 69807)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [8]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [9]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-3.5-turbo` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [13]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick based on the positive reviews provided in the context.'

In [14]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there is a review with a rating of 10.\n\nThe URL to that review is '/review/rw4854296/?ref_=tt_urv'."

In [15]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hit-man comes out of retirement to seek vengeance against the gangsters who killed his dog and took everything from him. The story is filled with lots of killings, thrills, noisy action, and suspense, with John Wick unleashing a maelstrom of destruction against those who come after him. The movie is known for its action-packed and gripping storyline.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [16]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [17]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [18]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Opinions on John Wick seem to vary. Some people really enjoyed the action and style of the movie, while others found it to be lacking in plot and substance. Overall, it seems that not everyone liked John Wick.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'I am sorry, but there are no reviews with a rating of 10 for the movie "John Wick 4" in the context provided.'

In [20]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'John Wick is a series of action movies featuring the character John Wick, played by Keanu Reeves. The movies are known for their intense action sequences and choreography.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [21]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [22]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, generally people liked John Wick based on the reviews provided.'

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, after resolving his issues with the Russian mafia, John Wick returns home only to be visited by mobster Santino D'Antonio, who asks Wick to kill his sister in Rome. When John completes this task, Santino puts a seven-million dollar contract on him, leading to a series of events where Wick must fight off professional killers to take down Santino."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [26]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [27]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick. The reviews praise the film for its slick action sequences, Keanu Reeves' performance, and the overall entertainment value it provides to viewers."

In [29]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'There is one review with a rating of 10. Here is the URL to that review: \n- /review/rw4854296/?ref_=tt_urv'

In [30]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In "John Wick," the main character John Wick, played by Keanu Reeves, is a retired assassin who comes out of retirement to seek revenge after someone kills his dog and steals his car. This leads to a series of intense and violent events involving assassins, the Russian mafia, and a lot of action scenes across different locations like Italy, Canada, and Manhattan. John Wick\'s quest for vengeance and subsequent involvement in the criminal underworld results in a high body count as he faces off against various adversaries.'

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [31]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [32]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/3m/gvjhqnvd0035_535_q0lxfkm0000gn/T/ipykernel_74457/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [33]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [34]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [35]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [36]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, opinions on John Wick are divided. Some people really enjoy the series and find it consistent and well-received, while others feel that it is a horrible movie with a nonsensical plot and boring fight scenes.'

In [37]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: /review/rw4854296/?ref_=tt_urv'

In [38]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick and its sequel, John Wick is a retired assassin who returns to his violent ways after his dog is killed and his car is stolen. In John Wick 2, he is forced back into the assassin world to pay off an old debt and faces off against numerous other assassins in Italy, Canada, and Manhattan.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [39]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [40]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [41]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Yes, people generally liked John Wick based on the reviews provided. The movie was well-received for its slick action sequences, Keanu Reeves' performance, and overall entertainment value."

In [42]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is one review with a rating of 10 for the movie "John Wick 3". Here is the URL to that review: \'/review/rw4854296/?ref_=tt_urv\'.'

In [43]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the movie "John Wick," an ex-hitman comes out of retirement to seek revenge on the gangsters who killed his dog and took everything from him. The movie features violent action, shootouts, and breathtaking fights as John Wick faces off against his enemies.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [44]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [45]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [46]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [47]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [48]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [49]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [50]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, it seems that generally, people enjoyed the John Wick movies. The majority of the reviewers had positive things to say about the action, the style, and the overall entertainment value of the films.'

In [51]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10 for the movie "John Wick 3." Here is the URL to that review: "/review/rw4854296/?ref_=tt_urv"'

In [52]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the main character seeks revenge on the people who took something he loved from him, which was his dog. It leads to a series of events involving action-packed scenes and stylish stunts as John Wick pursues his quest for vengeance.'

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

## Activity #1

### 1. Golden Dataset Creation
To compare different retriever methods, I created a synthetic “golden dataset” containing question–answer pairs. This dataset was generated (or curated) to ensure coverage of various question types and references.

### 2. Evaluating Each Retriever
I tested five retrievers—**BM25**, **Parent Document**, **Multi-Query**, **Compression**, and **Ensemble**—using **Ragas** metrics:

- **Context Recall** – Measures how much of the needed context is retrieved.  
- **Faithfulness** – Indicates how closely the answer aligns with the retrieved text.  
- **Factual Correctness (F1)** – Reflects how accurate the final answer is compared to the ground truth.

The results are as follows:

| Retriever          | Context Recall | Faithfulness | Factual Correctness (F1) |
|--------------------|---------------:|------------:|-------------------------:|
| **BM25**           |           0.80 |      0.7119 |                     0.484 |
| **Parent Document**|           0.80 |      0.6900 |                     0.358 |
| **Multi-Query**    |           0.85 |      0.9500 |                     0.284 |
| **Compression**    |           1.00 |      0.6850 |                     0.434 |
| **Ensemble**       |           1.00 |      0.8583 |                     0.296 |

### 3. Analysis (Cost, Latency, Performance)

1. **BM25**  
   - **Strengths**: High Factual Correctness (0.484), moderate Faithfulness (0.7119), decent Recall (0.80).  
   - **Trade-Offs**: Typically lower computational cost but may miss some nuances due to its sparse approach.

2. **Parent Document**  
   - **Strengths**: Solid overall Recall (0.80).  
   - **Trade-Offs**: Faithfulness (0.69) and Factual Correctness (0.358) lag behind BM25, suggesting it retrieves broader context but may not focus answers as effectively.

3. **Multi-Query**  
   - **Strengths**: Very high Faithfulness (0.95).  
   - **Trade-Offs**: Lower Factual Correctness (0.284), potentially due to the complexity of multiple queries. May also be **more expensive** and **slower** due to generating and retrieving with multiple prompts.

4. **Compression**  
   - **Strengths**: Perfect Context Recall (1.0), indicating it surfaces all relevant content.  
   - **Trade-Offs**: Faithfulness (0.685) and Factual Correctness (0.434) are respectable but not top-tier. Additional LLM passes can increase **cost** and **latency**.

5. **Ensemble**  
   - **Strengths**: Also achieves 1.0 Recall and strong Faithfulness (0.8583) by combining multiple methods.  
   - **Trade-Offs**: Factual Correctness (0.296) remains low, and ensembling can be **costly** and **complex**.

### 4. Conclusion
For this dataset, **BM25** provides the highest Factual Correctness (0.484), making it appealing if correctness is the primary goal. **Multi-Query** excels at Faithfulness (0.95) but suffers in correctness. **Compression** and **Ensemble** both achieve perfect Recall but have middling correctness. Ultimately, the best choice depends on whether you prioritize **accuracy**, **speed**, or **cost**.


##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [53]:
# Import required libraries
from ragas import evaluate, EvaluationDataset
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas.llms import LangchainLLMWrapper
from langchain.chat_models import ChatOpenAI

# Create dictionary of retrievers
retrievers = {
    "BM25": bm25_retriever,
    "Parent-Document": parent_document_retriever,
    "Multi-Query": multi_query_retriever,
    "Contextual Compression": compression_retriever,
    "Ensemble": ensemble_retriever
}

# Generate synthetic questions for evaluation
chat_model = ChatOpenAI(temperature=0.7)
golden_dataset = []

for doc in documents[:5]:  # Using first 5 documents for example
    question = chat_model.invoke(
        f"Based on this text, generate a specific question that tests understanding of the key information: {doc.page_content}"
    ).content
    
    answer = chat_model.invoke(
        f"Answer this question based on the text: {question}\n\nText: {doc.page_content}"
    ).content
    
    golden_dataset.append({
        "question": question,
        "ground_truth": answer
    })

/var/folders/3m/gvjhqnvd0035_535_q0lxfkm0000gn/T/ipykernel_74457/2167123061.py:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(temperature=0.7)


In [54]:
from ragas import evaluate, EvaluationDataset
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas.llms import LangchainLLMWrapper
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
evaluator_llm = LangchainLLMWrapper(chat_model)

In [55]:
import ast
from ragas import evaluate, EvaluationDataset
from ragas.metrics import ContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
evaluator_llm = LangchainLLMWrapper(chat_model)

bm25_dataset = []
for item in golden_dataset:
    query = item["question"]
    retrieved_docs = bm25_retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    response = chat_model.invoke(
        f"Question: {query}\nContext: {context}\nAnswer:"
    ).content
    bm25_dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": item["ground_truth"]
    })

evaluation_dataset = EvaluationDataset.from_list(bm25_dataset)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[ContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

# Assign the result to bm25_result so it can be used later in the comparison table.
bm25_result = result

print("BM25 EvaluationResult:")
print(bm25_result)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

BM25 EvaluationResult:
{'context_recall': 0.8000, 'faithfulness': 0.7119, 'factual_correctness(mode=f1)': 0.4840}


In [56]:
from ragas import evaluate, EvaluationDataset
from ragas.metrics import ContextRecall, Faithfulness, FactualCorrectness

parent_doc_dataset = []
for item in golden_dataset:
    query = item["question"]
    retrieved_docs = parent_document_retriever.invoke(query)
    
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    response = chat_model.invoke(f"Question: {query}\nContext: {context}\nAnswer:").content
    
    parent_doc_dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": item["ground_truth"]
    })

parent_doc_eval_dataset = EvaluationDataset.from_list(parent_doc_dataset)

parent_doc_result = evaluate(
    dataset=parent_doc_eval_dataset,
    metrics=[ContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

print("Parent Document Retriever Results:")
print(parent_doc_result)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Parent Document Retriever Results:
{'context_recall': 0.8000, 'faithfulness': 0.6900, 'factual_correctness(mode=f1)': 0.3580}


In [57]:
multi_query_dataset = []
for item in golden_dataset:
    query = item["question"]
    retrieved_docs = multi_query_retriever.invoke(query)
    
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    response = chat_model.invoke(f"Question: {query}\nContext: {context}\nAnswer:").content
    
    multi_query_dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": item["ground_truth"]
    })

multi_query_eval_dataset = EvaluationDataset.from_list(multi_query_dataset)

multi_query_result = evaluate(
    dataset=multi_query_eval_dataset,
    metrics=[ContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

print("Multi-Query Retriever Results:")
print(multi_query_result)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Multi-Query Retriever Results:
{'context_recall': 0.8500, 'faithfulness': 0.9500, 'factual_correctness(mode=f1)': 0.2840}


In [58]:
compression_dataset = []
for item in golden_dataset:
    query = item["question"]
    retrieved_docs = compression_retriever.invoke(query)
    
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    response = chat_model.invoke(f"Question: {query}\nContext: {context}\nAnswer:").content
    
    compression_dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": item["ground_truth"]
    })

compression_eval_dataset = EvaluationDataset.from_list(compression_dataset)

compression_result = evaluate(
    dataset=compression_eval_dataset,
    metrics=[ContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

print("Compression Retriever Results:")
print(compression_result)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Compression Retriever Results:
{'context_recall': 1.0000, 'faithfulness': 0.6850, 'factual_correctness(mode=f1)': 0.4340}


In [59]:
ensemble_dataset = []
for item in golden_dataset:
    query = item["question"]
    retrieved_docs = ensemble_retriever.invoke(query)
    
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    response = chat_model.invoke(f"Question: {query}\nContext: {context}\nAnswer:").content
    
    ensemble_dataset.append({
        "user_input": query,
        "retrieved_contexts": [doc.page_content for doc in retrieved_docs],
        "response": response,
        "reference": item["ground_truth"]
    })

ensemble_eval_dataset = EvaluationDataset.from_list(ensemble_dataset)

ensemble_result = evaluate(
    dataset=ensemble_eval_dataset,
    metrics=[ContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

print("Ensemble Retriever Results:")
print(ensemble_result)


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Ensemble Retriever Results:
{'context_recall': 1.0000, 'faithfulness': 0.8583, 'factual_correctness(mode=f1)': 0.2960}


In [60]:
import pandas as pd
import ast

# List of (retriever name, evaluation result) pairs.
eval_results = [
    ("BM25", bm25_result),
    ("Parent Document", parent_doc_result),
    ("Multi-Query", multi_query_result),
    ("Compression", compression_result),
    ("Ensemble", ensemble_result),
]

# Convert each result's string representation to a real dict and build the comparison table.
comparison_data = []
for name, res in eval_results:
    res_dict = ast.literal_eval(str(res))
    comparison_data.append({
        "Retriever": name,
        "Context Recall": res_dict.get("context_recall"),
        "Faithfulness": res_dict.get("faithfulness"),
        "Factual Correctness (F1)": res_dict.get("factual_correctness(mode=f1)")
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df


,Retriever,Context Recall,Faithfulness,Factual Correctness (F1)
0,BM25,0.80,0.7119,0.484
1,Parent Document,0.80,0.6900,0.358
2,Multi-Query,0.85,0.9500,0.284
3,Compression,1.00,0.6850,0.434
4,Ensemble,1.00,0.8583,0.296
